Lambda School Data Science

*Unit 1, Sprint 1, Module 4*

---

# ASSIGNMENT

### 1) Replicate the lesson code. I recommend that you [do not copy-paste](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit).

Get caught up to where we got our example in class and then try and take things further. How close to "pixel perfect" can you make the lecture graph?

Once you have something that you're proud of, share your graph in the cohort channel and move on to the second exercise.

### 2) Reproduce another example from [FiveThityEight's shared data repository](https://data.fivethirtyeight.com/).

**WARNING**: There are a lot of very custom graphs and tables at the above link. I **highly** recommend not trying to reproduce any that look like a table of values or something really different from the graph types that we are already familiar with. Search through the posts until you find a graph type that you are more or less familiar with: histogram, bar chart, stacked bar chart, line chart, [seaborn relplot](https://seaborn.pydata.org/generated/seaborn.relplot.html), etc. Recreating some of the graphics that 538 uses would be a lot easier in Adobe photoshop/illustrator than with matplotlib. 

- If you put in some time to find a graph that looks "easy" to replicate you'll probably find that it's not as easy as you thought. 

- If you start with a graph that looks hard to replicate you'll probably run up against a brick wall and be disappointed with your afternoon.












In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
!wget trump-approval-ratings.zip

--2020-03-20 01:46:32--  http://trump-approval-ratings.zip/
Resolving trump-approval-ratings.zip (trump-approval-ratings.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘trump-approval-ratings.zip’


In [5]:
!unzip trump-approval-ratings.zip

Archive:  trump-approval-ratings.zip
   creating: trump-approval-ratings/
  inflating: trump-approval-ratings/README.md  
  inflating: trump-approval-ratings/approval_polllist.csv  
  inflating: trump-approval-ratings/approval_topline.csv  


In [11]:
poll_df = pd.read_csv('trump-approval-ratings/approval_polllist.csv')
poll_df.head(5)

,president,subgroup,modeldate,startdate,enddate,pollster,grade,samplesize,population,weight,influence,approve,disapprove,adjusted_approve,adjusted_disapprove,multiversions,tracking,url,poll_id,question_id,createddate,timestamp
0,Donald Trump,All polls,3/19/2020,1/20/2017,1/22/2017,Gallup,B,1500.0,a,0.241682,0.0,45.0,45.0,45.725344,43.596084,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49253,77265,1/23/2017,20:13:21 19 Mar 2020
1,Donald Trump,All polls,3/19/2020,1/20/2017,1/22/2017,Morning Consult,B/C,1992.0,rv,0.696984,0.0,46.0,37.0,44.733308,38.303600,NaN,NaN,http://static.politico.com/9b/13/82a3baf542ae9...,49249,77261,1/23/2017,20:13:21 19 Mar 2020
2,Donald Trump,All polls,3/19/2020,1/21/2017,1/23/2017,Gallup,B,1500.0,a,0.223234,0.0,45.0,46.0,45.725344,44.596084,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49262,77274,1/24/2017,20:13:21 19 Mar 2020
3,Donald Trump,All polls,3/19/2020,1/20/2017,1/24/2017,Ipsos,B-,1632.0,a,0.150785,0.0,42.1,45.2,42.910255,44.036629,NaN,T,http://polling.reuters.com/#poll/CP3_2/,49426,77599,3/1/2017,20:13:21 19 Mar 2020
4,Donald Trump,All polls,3/19/2020,1/22/2017,1/24/2017,Rasmussen Reports/Pulse Opinion Research,C+,1500.0,lv,0.200335,0.0,57.0,43.0,51.202481,44.632434,NaN,T,http://www.rasmussenreports.com/public_content...,49266,77278,1/25/2017,20:13:21 19 Mar 2020


# STRETCH OPTIONS

### 1) Reproduce one of the following using the matplotlib or seaborn libraries:

- [thanksgiving-2015](https://fivethirtyeight.com/features/heres-what-your-part-of-america-eats-on-thanksgiving/) 
- [candy-power-ranking](https://fivethirtyeight.com/features/the-ultimate-halloween-candy-power-ranking/) 
- or another example of your choice!

### 2) Make more charts!

Choose a chart you want to make, from [Visual Vocabulary - Vega Edition](http://ft.com/vocabulary).

Find the chart in an example gallery of a Python data visualization library:
- [Seaborn](http://seaborn.pydata.org/examples/index.html)
- [Altair](https://altair-viz.github.io/gallery/index.html)
- [Matplotlib](https://matplotlib.org/gallery.html)
- [Pandas](https://pandas.pydata.org/pandas-docs/stable/visualization.html)

Reproduce the chart. [Optionally, try the "Ben Franklin Method."](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit) If you want, experiment and make changes.

Take notes. Consider sharing your work with your cohort!

In [0]:
# More Work Here